In [1]:
import pandas as pd
import re,json

# 程凯json处理

In [2]:
with open('imagine_link_new.json','r',encoding = 'gbk') as f:
    f = f.read()
file = json.loads(f)
data = pd.DataFrame(file)
data = data.stack().reset_index()
data.columns = ['关键词','联想词','value']
data.head()

,关键词,联想词,value
0,EMAIL,保险公司,"[1.125, 1]"
1,EMAIL,信用卡,"[3.115, 2]"
2,EMAIL,出票,"[1.787, 1]"
3,EMAIL,国泰,"[1.351, 1]"
4,EMAIL,收据,"[3.6319999999999997, 2]"


In [3]:
data['频次'] = data.value.str[1]
data['threshold'] = data.value.str[0]/data['频次']
data.head()

,关键词,联想词,value,频次,threshold
0,EMAIL,保险公司,"[1.125, 1]",1,1.1250
1,EMAIL,信用卡,"[3.115, 2]",2,1.5575
2,EMAIL,出票,"[1.787, 1]",1,1.7870
3,EMAIL,国泰,"[1.351, 1]",1,1.3510
4,EMAIL,收据,"[3.6319999999999997, 2]",2,1.8160


In [4]:
cols = list(data.columns)
missing = list(map(lambda i:data[i].isnull().sum(),cols))
missing_dict = dict(zip(cols,missing))
missing_dict

{'threshold': 0, 'value': 0, '关键词': 0, '联想词': 0, '频次': 0}

In [5]:
data[['关键词','联想词','threshold','频次']].to_csv('关键词联想词_0816.txt',sep = '\t',encoding = 'utf-8',index = False)

# 关键词情绪处理

In [6]:
dfa = pd.read_table('tianbao_result_keyword_new.txt',sep = '\t',error_bad_lines=False)
dfa.head()

,ID,date,phone,type,sentence,emotion,threshold,keyword,area
0,1,20170624,6475223313,1,嗯,NaN,0.000,NaN,Ontario
1,1,20170624,6475223313,1,喂你好，,NaN,0.000,NaN,Ontario
2,1,20170624,6475223313,2,你好，我是珍惜,NaN,0.000,NaN,Ontario
3,1,20170624,6475223313,2,刚才打电话给你啊你好，啊那个胡玉成有没有早上的飞机呢？,NaN,0.000,NaN,Ontario
4,1,20170624,6475223313,1,早上啊看一下，对,愤怒,1.123,NaN,Ontario


In [7]:
cols  = list(dfa.columns)
missing = list(map(lambda i :dfa[i].isnull().sum(),cols))
missing_dict = dict(zip(cols,missing))
missing_dict

{'ID': 0,
 'area': 0,
 'date': 0,
 'emotion': 13504,
 'keyword': 21936,
 'phone': 0,
 'sentence': 0,
 'threshold': 0,
 'type': 0}

In [8]:
dfa.area.value_counts()

Ontario                                 20656
unkown                                   2139
Quebec                                   1403
British Columbia                          746
Alberta                                   407
Manitoba                                   39
New Brunswick                              28
Nova Scotia and Prince Edward Island       20
Name: area, dtype: int64

## 更改错别字

In [9]:
dfa.loc[dfa.area == 'unkown','area'] = 'unknown'

In [10]:
dfa.area.value_counts()

Ontario                                 20656
unknown                                  2139
Quebec                                   1403
British Columbia                          746
Alberta                                   407
Manitoba                                   39
New Brunswick                              28
Nova Scotia and Prince Edward Island       20
Name: area, dtype: int64

In [11]:
dfa['country'] = 'Canada'

In [12]:
dfa.columns

Index(['ID', 'date', 'phone', 'type', 'sentence', 'emotion', 'threshold',
       'keyword', 'area', 'country'],
      dtype='object')

In [13]:
dfa.to_csv('geo.txt',sep = '\t',index = False,encoding = 'utf-8')

## 关键词同义词合并

In [14]:
dfa.keyword.value_counts()

机票       619
出票       366
信用卡      246
机场       219
改签       196
行程       184
直飞       163
积分       156
座位       153
转机       136
行李       132
EMAIL    121
退票        85
公斤        78
海航        76
仓位        66
东航        65
托运        64
费用        63
团费        52
手续费       40
罚款        34
保险公司      25
国泰        24
罚金        20
海南航空      18
参团        16
收据        16
东方航空      14
退款        13
保单        12
罚钱         8
保额         6
租车         4
医疗保险       4
拼房         4
服务费        4
Name: keyword, dtype: int64

In [15]:
dfa['keyword'].replace(['东航','海航','退票','罚金','罚钱'],['东方航空','海南航空','退款','罚款','罚款'],inplace=True)

In [32]:
pd.DataFrame(dfa.keyword.value_counts()).to_csv('keywords_frequency_new.txt',sep = '\t',encoding = 'utf-8')

In [17]:
dfa.to_csv('天宝旅游关键词情绪_0816.txt',encoding = 'utf-8',sep = '\t',index = False)

# 带关键词的用户数据

In [18]:
customer= dfa.loc[dfa.type == 1]
cond = customer.keyword.notnull()
customer = customer.loc[cond].copy()
customer.shape

(1959, 10)

In [19]:
customer.head(10)

,ID,date,phone,type,sentence,emotion,threshold,keyword,area,country
23,2,20170704,6476695236,1,唉没有，我联系一下，对礼拜六礼拜天你帮我看下去福州的，我老婆机票也有可能，我也感觉因为今天刚...,愤怒,1.538,机票,Ontario,Canada
56,2,20170704,6476695236,1,8号的凌晨，那其他那个噢你你们最近怎么跟东航南航关系比较好，都是往南走。吧。,愤怒,2.116,东方航空,Ontario,Canada
65,2,20170704,6476695236,1,东航呢4点半,愤怒,0.979,东方航空,Ontario,Canada
66,2,20170704,6476695236,1,噢东航东航4点半的对下午4点半的唉是到哪里了？,愤怒,1.034,东方航空,Ontario,Canada
78,2,20170704,6476695236,1,还有这样子，现在买机票了,愤怒,0.975,机票,Ontario,Canada
82,2,20170704,6476695236,1,啊广州转机啊,焦虑,1.007,转机,Ontario,Canada
108,3,20170710,6479959904,1,唉你好唉，你好，姚老板那里，我问一下那个退票汇票啊陈道明给我一下吗？,NaN,0.000,退款,Ontario,Canada
125,3,20170710,6479959904,1,转账啊当然退票多man。,愤怒,3.041,退款,Ontario,Canada
137,3,20170710,6479959904,1,你们岂不是要交手续费？这样子搞。,厌恶,1.723,手续费,Ontario,Canada
255,6,20170712,6478089169,1,但是它还是按照改票费用来算的，我当然它就只改这一段，它至少都要补300块钱。我因为车票的改签...,愤怒,1.423,改签,Ontario,Canada


In [20]:
customer.to_csv('带关键词的用户数据_0816.txt',encoding = 'utf-8',index = False,sep = '\t')

In [21]:
grouped = customer.groupby('keyword')['ID'].count()
people = pd.DataFrame(grouped.sort_values(ascending =False))
people.head()

,ID
keyword,
机票,309
出票,210
机场,137
信用卡,133
改签,132


In [22]:
people.to_csv('关键词提及人数统计_0816.txt',sep = '\t',encoding = 'utf-8')

# 关键词词频处理

In [23]:
frequency = pd.read_table('keywords_frequency.txt',sep = '\t')
frequency.head()

,keyword,词频
0,邮箱,60
1,签证,136
2,手机,82
3,改签,107
4,海航,85


In [24]:
dfb = dfa[['emotion','threshold','keyword','sentence']].copy()
dfb.head()

,emotion,threshold,keyword,sentence
0,NaN,0.000,NaN,嗯
1,NaN,0.000,NaN,喂你好，
2,NaN,0.000,NaN,你好，我是珍惜
3,NaN,0.000,NaN,刚才打电话给你啊你好，啊那个胡玉成有没有早上的飞机呢？
4,愤怒,1.123,NaN,早上啊看一下，对


In [25]:
dfb = pd.DataFrame(dfb.groupby('keyword').threshold.mean())
dfb.head()

,threshold
keyword,
EMAIL,0.649669
东方航空,1.067848
仓位,1.062924
保单,1.119417
保险公司,1.105520


In [26]:
dfb = dfa[['emotion','threshold','keyword','sentence']].copy()
dfb = pd.DataFrame(dfb.groupby('keyword').threshold.mean())
dfb.head()

,threshold
keyword,
EMAIL,0.649669
东方航空,1.067848
仓位,1.062924
保单,1.119417
保险公司,1.105520


In [27]:
dfb.to_csv('keyword_thresh_0816.txt',sep ='\t',encoding = 'utf-8')

In [28]:
dfc = pd.read_table('keyword_thresh.txt')
dfc.head()

,keyword,threshold
0,EMAIL,0.283893
1,QQ,0.375427
2,wifi,0.289720
3,东方航空,0.854357
4,东航,0.590446


In [29]:
dfd = pd.merge(frequency[['keyword','词频']],dfc)
dfd.head()

,keyword,词频,threshold
0,邮箱,60,0.229984
1,签证,136,0.372948
2,手机,82,0.411692
3,改签,107,0.333408
4,海航,85,0.518408


In [30]:
dfd.to_csv('关键词词频情绪_0816.txt',sep = ',',encoding = 'utf-8')